<a href="https://colab.research.google.com/github/dalloliogm/dream2025/blob/main/DREAM2035_AutoGluon_Submission_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DREAM Target 2035: Step 1 Model – Multi-Fingerprint + Imbalanced Validation + Submission

In [1]:
!pip install -q autogluon pandas numpy pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See

In [1]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

## Load and Prepare Training Data

In [5]:
!ls data

Step1_TestData_Target2035.parquet


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
df = pd.read_parquet('drive/MyDrive/dream2035/WDR91.parquet')
fps = ['ECFP4', 'ECFP6', 'FCFP4', 'FCFP6', 'MACCS', 'RDK', 'AVALON', 'ATOMPAIR', 'TOPTOR']
X = np.hstack([np.stack(df[fp].values) for fp in fps])
y = df['TARGET_VALUE'].values

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [ ]:
df.head()

## Create Imbalanced Validation Set (~0.4% active)

In [ ]:
# Separate actives/inactives
X_pos, X_neg = X[y == 1], X[y == 0]
y_pos, y_neg = y[y == 1], y[y == 0]
n_val_pos = min(500, len(X_pos))
n_val_neg = n_val_pos * 250
X_val_pos, _, y_val_pos, _ = train_test_split(X_pos, y_pos, test_size=(1 - n_val_pos / len(X_pos)), random_state=42)
X_val_neg, _, y_val_neg, _ = train_test_split(X_neg, y_neg, test_size=(1 - n_val_neg / len(X_neg)), random_state=42)
X_val = np.vstack([X_val_pos, X_val_neg])
y_val = np.hstack([y_val_pos, y_val_neg])
X_train = np.vstack([X_pos, X_neg])
y_train = np.hstack([y_pos, y_neg])
train_df = pd.DataFrame(X_train)
train_df['Activity'] = y_train
val_df = pd.DataFrame(X_val)
val_df['Activity'] = y_val

## Train AutoGluon Model

In [ ]:
predictor = TabularPredictor(label='Activity', eval_metric='roc_auc').fit(train_df, time_limit=600)

## Load Test Set and Generate Submission

In [ ]:
test_df = pd.read_parquet('data/Step1_TestData_Target2035.parquet')
X_test = np.hstack([np.stack(test_df[fp].values) for fp in fps])
X_test_df = pd.DataFrame(X_test)
random_ids = test_df['RandomID'].values

In [ ]:
# Predict scores
scores = predictor.predict_proba(X_test_df)[1]
submission = pd.DataFrame({'RandomID': random_ids, 'Score': scores})
submission = submission.sort_values('Score', ascending=False).reset_index(drop=True)
submission['Sel_200'] = 0
submission['Sel_200'].iloc[:200] = 1
submission['Sel_500'] = 0
submission['Sel_500'].iloc[:500] = 1
submission = submission[['RandomID', 'Sel_200', 'Sel_500', 'Score']]
submission.to_csv('TeamMyTeamName.csv', index=False)
submission.head()